# "Blog - migrate to quarto"
> "following migration guide from Hamel Hussain"
- show_tags: true
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [nbdev, quarto, fastpages]
- image: images/icons/fastai.png

# Inspiration

Hamel just announced that [fastpages will be discontinued](https://forums.fast.ai/t/fastpages-deprecating-fastpages-in-favor-of-quarto/99095) as nbdev+quarto is now a valid option to provide a blogging platform.
He has written a [migration guide](https://nbdev.fast.ai/tutorials/blogging.html) for that. 

This is my walkthrough.


# Walkthrough